# Top2Vec

In [ ]:
!pip install top2vec
!pip install top2vec[sentence_encoders]
!pip install top2vec[sentence_transformers]
!pip install top2vec[indexing]
!pip install tensorflow tensorflow_hub tensorflow_text

In [ ]:
import multiprocessing
import sys
import warnings
from ast import literal_eval

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from top2vec import Top2Vec

warnings.filterwarnings("ignore")
np.set_printoptions(threshold=sys.maxsize)

%matplotlib inline

In [ ]:
data = pd.read_csv('parsed_data_train.csv', index_col=0)
raex_list = pd.read_excel("RAEX2024.xlsx")

data = pd.merge(data, raex_list[['Название', '№']], left_on='company', right_on='Название', how='left')
data = data.rename(columns={'№': 'rating'})
data.drop(['Название'], axis=1, inplace=True)
data = data[['rating'] + [col for col in data if col != 'rating']]

data['text_cyr'] = data['text'].str.replace(r"[^а-яА-Я\s]+", '', regex=True).str.strip()
data['text_cyr_words'] = data['text_cyr'].str.split()
data = data.replace(r'^s*$', float('NaN'), regex=True).dropna().reset_index(drop=True)
data['rating'] = data['rating'].astype('str')
data

,rating,company,url,text,text_cyr,text_cyr_words
0,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,обращать внимание мошеннический схема ао кауст...,обращать внимание мошеннический схема ао кауст...,"[обращать, внимание, мошеннический, схема, ао,..."
1,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,ао каустик г волгоград,ао каустик г волгоград,"[ао, каустик, г, волгоград]"
2,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,служба поддержка клиент тело факс,служба поддержка клиент тело факс,"[служба, поддержка, клиент, тело, факс]"
3,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,email sbyturkaustikru контакт управление реали...,контакт управление реализация,"[контакт, управление, реализация]"
4,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,главный,главный,[главный]
...,...,...,...,...,...,...
1076178,50,Якутская топливно-энергетическая компания (ЯТЭК),https://www.yatec.ru/press-centr/photogallery/,пао якутский топливноэнергетический компания,пао якутский топливноэнергетический компания,"[пао, якутский, топливноэнергетический, компания]"
1076179,50,Якутская топливно-энергетическая компания (ЯТЭК),https://www.yatec.ru/press-centr/photogallery/,правовой информация,правовой информация,"[правовой, информация]"
1076180,50,Якутская топливно-энергетическая компания (ЯТЭК),https://www.yatec.ru/press-centr/photogallery/,положение конфиденциальность,положение конфиденциальность,"[положение, конфиденциальность]"
1076181,50,Якутская топливно-энергетическая компания (ЯТЭК),https://www.yatec.ru/press-centr/photogallery/,политика использование cookies,политика использование,"[политика, использование]"


In [ ]:
%%time
umap_args = {"low_memory": True,
             "random_state": 42}
docs = np.unique(data['text_cyr'].values)

model = Top2Vec(
    documents=docs,
    embedding_model='universal-sentence-encoder-multilingual',
    speed='deep-learn',
    workers=8,
    min_count=0,
    verbose=True,
    umap_args=umap_args
)

2024-05-25 14:01:34,784 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
2024-05-25 14:01:55,961 - top2vec - INFO - Downloading universal-sentence-encoder-multilingual model
INFO:top2vec:Downloading universal-sentence-encoder-multilingual model
2024-05-25 14:02:07,616 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2024-05-25 14:19:22,760 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2024-05-25 14:27:10,230 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2024-05-25 14:27:27,762 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


CPU times: user 33min 12s, sys: 2min 28s, total: 35min 41s
Wall time: 26min 31s


In [ ]:
model.get_num_topics()

1267

In [ ]:
%%time
topic_mapping = model.hierarchical_topic_reduction(num_topics=60)

CPU times: user 26min 26s, sys: 2min 8s, total: 28min 35s
Wall time: 19min 18s


In [ ]:
print(model.get_num_topics(reduced=False))
print(model.get_num_topics(reduced=True))

1267
60


In [ ]:
topics_words, word_scores, topic_nums = model.get_topics(reduced=True)
topics_words, word_scores, topic_nums

(array([['раичихинскии', 'улмихалковскии', 'юлмалоохтинскии',
         'захарьинскии', 'тимирязевскиига', 'якшурбодьинскии',
         'улкомсомольскии', 'пякяхинскии', 'волчихинскии',
         'екатерининскии', 'алмаатинскии', 'картебунинскии',
         'сакрушинскии', 'вынгаяхинскии', 'калининскии', 'шелепихинскии',
         'раичихинск', 'верхнеуральск', 'ангароилимскии', 'алтаесаянск',
         'нежданинскии', 'алтуховкоманда', 'петелинскии',
         'ульяновскнефть', 'уллуначарскии', 'ульяновскг', 'еманжелинск',
         'тольяттинскии', 'заикинскии', 'улбурмакинскии', 'раичихнск',
         'нижнемазинскии', 'устьудинскии', 'еаланшаков', 'халтуринскии',
         'верхнеимбатск', 'гмосквадругии', 'качалинскии', 'ухтинскии',
         'носовихинскии', 'ивакинскии', 'тортасинскии', 'сахасахалинскии',
         'лаврушинскии', 'наталскиинскии', 'канскоачинскии',
         'марушинскии', 'алтаесаянскии', 'лахтинскии', 'большеохтинскии'],
        ['банкекредитныи', 'кредитабанк', 'сахакред

In [ ]:
topics_raw_data = pd.DataFrame({'topic_num': topic_nums, 'topics_words': list(topics_words)})
topics_raw_data.head()

,topic_num,topics_words
0,0,"[раичихинскии, улмихалковскии, юлмалоохтинскии..."
1,1,"[банкекредитныи, кредитабанк, сахакредитбанк, ..."
2,2,"[пон, гыд, ыд, тмм, ич, ек, жижа, атво, тпу, в..."
3,3,"[москвемосква, регионахсибирьо, улбурмакинскии..."
4,4,"[развитииновость, развитияпочесть, развитиенов..."


In [ ]:
topics_raw_data.to_excel('topics_raw_data.xlsx', index=False)

# Collecting corpora of words by company

In [ ]:
data.head()

,rating,company,url,text,text_cyr,text_cyr_words
0,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,обращать внимание мошеннический схема ао кауст...,обращать внимание мошеннический схема ао кауст...,"[обращать, внимание, мошеннический, схема, ао,..."
1,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,ао каустик г волгоград,ао каустик г волгоград,"[ао, каустик, г, волгоград]"
2,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,служба поддержка клиент тело факс,служба поддержка клиент тело факс,"[служба, поддержка, клиент, тело, факс]"
3,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,email sbyturkaustikru контакт управление реали...,контакт управление реализация,"[контакт, управление, реализация]"
4,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,главный,главный,[главный]


In [ ]:
%%time
data_company = data.groupby(['company', 'rating'])[['text_cyr_words']].apply(sum).reset_index().sort_values(by=['company', 'rating'])
data_company['text_cyr_words'] = data_company['text_cyr_words'].agg(pd.unique)

CPU times: user 22min 16s, sys: 13.4 s, total: 22min 30s
Wall time: 23min 28s


In [ ]:
data_company

,company,rating,text_cyr_words
0,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,..."
1,"""Нефтиса"", нефтяная компания",144-145,"[охрана, окружающий, среда, общий, вопрос, тел..."
2,"""Соликамскбумпром""",86,"[резинотехнический, изделие, компания, история..."
3,"""Титан"", группа компаний (деревообработка)",140,"[устойчивый, развитие, группа, компания, титан..."
4,En+ Group,26,"[тендер, эн, компания, заниматься, инвестор, у..."
...,...,...,...
115,ТГК-2,111,"[политика, область, охрана, труд, личный, каби..."
116,ТМК (Трубная металлургическая компания),44,"[миссия, стратегия, тмк, продажа, поставщик, к..."
117,Транспортная группа FESCO,89,"[транспортный, группа, инновация, лидерство, о..."
118,Южуралзолото Группа Компаний,138,"[компания, наш, цель, история, руководство, ко..."


In [ ]:
data_company.to_csv('parsed_data_company_train.csv', encoding='utf-8')

Topic selection

In [ ]:
topics_data = pd.read_excel('topics_raw_data.xlsx')
topics_data['topics_words'] = topics_data['topics_words'].apply(lambda x: literal_eval(x.replace(' ', ','))).tolist()
topics_data

,topic_num,topics_words
0,0,"[раичихинскии, улмихалковскии, юлмалоохтинскии..."
1,1,"[банкекредитныи, кредитабанк, сахакредитбанк, ..."
2,2,"[пон, гыд, ыд, тмм, ич, ек, жижа, атво, тпу, в..."
3,3,"[москвемосква, регионахсибирьо, улбурмакинскии..."
4,4,"[развитииновость, развитияпочесть, развитиенов..."
5,5,"[суммывнесение, суммыполучение, суммупополнять..."
6,6,"[строикомбинат, строиплощадка, строиурал, стро..."
7,7,"[годомобеспечить, летежегодныи, годомпочта, еж..."
8,8,"[безопасностьва, безопасность, безопасноешаг, ..."
9,9,"[мчкпплощадка, прсююмбик, подлокотник, гофроко..."


In [ ]:
report_data = pd.read_csv('esg_words.csv', index_col=0)
report_data['words'] = report_data['words'].apply(lambda x: x.split(', '))
report_data = report_data.groupby('topic_name')['words'].apply(lambda x: list(set(sum(x, [])))).reset_index()
report_data

,topic_name,words
0,E,"[утилизация, энергоснабжение, эко, лесничество..."
1,Антикоррупция,"[электропрофсоюз, ликвидность, меняться, курс,..."
2,Безопасность и охрана здоровья,"[пациент, ликвидность, страховщик, трудность, ..."
3,Безопасность продукта,"[ликвидность, финансовыи, функция, характеризо..."
4,Биоразнообразие,"[эко, земля, граница, онкологическии, антропог..."
5,Благотворительность,"[хоккеиныи, биография, искусство, спектакль, к..."
6,Вода,"[утилизация, гидроагрегат, поселение, водопров..."
7,Газ,"[нефтешламовыи, топливо, нефтехим, севергазпро..."
8,Дивиденды и акционеры,"[исполнительныи, закладывать, амортизация, под..."
9,Инвестиции и капитальные вложения,"[закладывать, ликвидность, бизнес, замосковныи..."


In [ ]:
topics = []
for index, row in topics_data.iterrows():
    tf_idf = TfidfVectorizer()
    score = 0
    text_topics_words = np.array([elm for elm in row['topics_words'] if isinstance(elm, str)])
    for index2, row2 in report_data.iterrows():
        report_topics_words = np.array([elm for elm in row2['words'] if isinstance(elm, str)])

        text_topics_words = [' '.join(text_topics_words)]
        report_topics_words = [' '.join(report_topics_words)]

        data_company_tf = tf_idf.fit_transform(text_topics_words)
        topics_data_tf = tf_idf.transform(report_topics_words)

        cosine_similarity = linear_kernel(data_company_tf, topics_data_tf).flatten()
        if cosine_similarity[0]/len(row2['words']) > score:
            score = cosine_similarity[0]/len(row2['words'])
            topic_name = row2['topic_name']
    topics.append(topic_name)

topics_data.insert(0, 'topic_name', topics)
topics_data.to_csv('topics_data.csv', encoding='utf-8')
topics_data

,topic_name,topic_num,topics_words
0,Безопасность продукта,0,"[раичихинскии, улмихалковскии, юлмалоохтинскии..."
1,Безопасность продукта,1,"[банкекредитныи, кредитабанк, сахакредитбанк, ..."
2,Безопасность продукта,2,"[пон, гыд, ыд, тмм, ич, ек, жижа, атво, тпу, в..."
3,Биоразнообразие,3,"[москвемосква, регионахсибирьо, улбурмакинскии..."
4,Биоразнообразие,4,"[развитииновость, развитияпочесть, развитиенов..."
5,Биоразнообразие,5,"[суммывнесение, суммыполучение, суммупополнять..."
6,Отходы,6,"[строикомбинат, строиплощадка, строиурал, стро..."
7,Энергия,7,"[годомобеспечить, летежегодныи, годомпочта, еж..."
8,Безопасность продукта,8,"[безопасностьва, безопасность, безопасноешаг, ..."
9,Безопасность продукта,9,"[мчкпплощадка, прсююмбик, подлокотник, гофроко..."


# Assigning topics to companies

In [ ]:
data_company = pd.read_csv('parsed_data_company_train.csv', index_col=0)
data_company['text_cyr_words'] = data_company['text_cyr_words'].apply(lambda x: literal_eval(x.replace(' ', ','))).tolist()
data_company

,company,rating,text_cyr_words
0,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,..."
1,"""Нефтиса"", нефтяная компания",144-145,"[охрана, окружающий, среда, общий, вопрос, тел..."
2,"""Соликамскбумпром""",86,"[резинотехнический, изделие, компания, история..."
3,"""Титан"", группа компаний (деревообработка)",140,"[устойчивый, развитие, группа, компания, титан..."
4,En+ Group,26,"[тендер, эн, компания, заниматься, инвестор, у..."
...,...,...,...
115,ТГК-2,111,"[политика, область, охрана, труд, личный, каби..."
116,ТМК (Трубная металлургическая компания),44,"[миссия, стратегия, тмк, продажа, поставщик, к..."
117,Транспортная группа FESCO,89,"[транспортный, группа, инновация, лидерство, о..."
118,Южуралзолото Группа Компаний,138,"[компания, наш, цель, история, руководство, ко..."


In [ ]:
topics_companies_data = data_company.merge(topics_data, how='cross')
topics_companies_data

,company,rating,text_cyr_words,topic_name,topic_num,topics_words
0,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,...",Безопасность продукта,0,"[раичихинскии, улмихалковскии, юлмалоохтинскии..."
1,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,...",Безопасность продукта,1,"[банкекредитныи, кредитабанк, сахакредитбанк, ..."
2,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,...",Безопасность продукта,2,"[пон, гыд, ыд, тмм, ич, ек, жижа, атво, тпу, в..."
3,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,...",Биоразнообразие,3,"[москвемосква, регионахсибирьо, улбурмакинскии..."
4,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,...",Биоразнообразие,4,"[развитииновость, развитияпочесть, развитиенов..."
...,...,...,...,...,...,...
7195,Якутская топливно-энергетическая компания (ЯТЭК),50,"[акция, раздел, разработка, новый, версия, офи...",Отходы,55,"[ремонтник, ремонтвоити, ремонта, ремонтускидк..."
7196,Якутская топливно-энергетическая компания (ЯТЭК),50,"[акция, раздел, разработка, новый, версия, офи...",Безопасность продукта,56,"[годфинансовыи, годафинансовыи, годуфинансовыи..."
7197,Якутская топливно-энергетическая компания (ЯТЭК),50,"[акция, раздел, разработка, новый, версия, офи...",Обучение и развитие,57,"[обучениязабота, обучениедля, обучениявысокии,..."
7198,Якутская топливно-энергетическая компания (ЯТЭК),50,"[акция, раздел, разработка, новый, версия, офи...",Лидерство,58,"[авиаперевозчик, авиазаправочныи, авиаперевозк..."


In [ ]:
%%time
cosine_similarities = []
for index, row in topics_companies_data.iterrows():
    tf_idf = TfidfVectorizer()

    text_cyr_words = np.array([elm for elm in row['text_cyr_words'] if isinstance(elm, str)])
    topics_words = np.array([elm for elm in row['topics_words'] if isinstance(elm, str)])

    text_cyr_words = [' '.join(text_cyr_words)]
    topics_words = [' '.join(topics_words)]

    data_company_tf = tf_idf.fit_transform(text_cyr_words)
    topics_data_tf = tf_idf.transform(topics_words)

    cosine_similarity = linear_kernel(data_company_tf, topics_data_tf).flatten()
    cosine_similarities.append(cosine_similarity[0])

CPU times: user 2min 16s, sys: 495 ms, total: 2min 16s
Wall time: 2min 18s


In [ ]:
topics_companies_data['cosine_similarity'] = cosine_similarities
topics_companies_data.head()

,company,rating,text_cyr_words,topic_name,topic_num,topics_words,cosine_similarity
0,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,...",Безопасность продукта,0,"[раичихинскии, улмихалковскии, юлмалоохтинскии...",0.000000
1,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,...",Безопасность продукта,1,"[банкекредитныи, кредитабанк, сахакредитбанк, ...",0.000000
2,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,...",Безопасность продукта,2,"[пон, гыд, ыд, тмм, ич, ек, жижа, атво, тпу, в...",0.034638
3,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,...",Биоразнообразие,3,"[москвемосква, регионахсибирьо, улбурмакинскии...",0.000000
4,"""Каустик"" (г. Волгоград)",60,"[обращать, внимание, мошеннический, схема, ао,...",Биоразнообразие,4,"[развитииновость, развитияпочесть, развитиенов...",0.000000


In [ ]:
features_data = (
    topics_companies_data
    .pivot(index=['company', 'rating'], columns='topic_num', values='cosine_similarity')
    .reset_index()
    .add_prefix('topic_num_')
    .rename(columns={'topic_num_company': 'company', 'topic_num_rating': 'ESG_Rank'})
)

features_data.head()

topic_num,company,ESG_Rank,topic_num_0,topic_num_1,topic_num_2,topic_num_3,topic_num_4,topic_num_5,topic_num_6,topic_num_7,...,topic_num_50,topic_num_51,topic_num_52,topic_num_53,topic_num_54,topic_num_55,topic_num_56,topic_num_57,topic_num_58,topic_num_59
0,"""Каустик"" (г. Волгоград)",60,0.0,0.0,0.034638,0.0,0.000000,0.0,0.018972,0.015490,...,0.032860,0.034638,0.026830,0.0,0.015490,0.028980,0.010953,0.026830,0.010953,0.028980
1,"""Нефтиса"", нефтяная компания",144-145,0.0,0.0,0.030994,0.0,0.000000,0.0,0.030994,0.030994,...,0.057984,0.030994,0.043832,0.0,0.021916,0.030994,0.000000,0.030994,0.000000,0.030994
2,"""Соликамскбумпром""",86,0.0,0.0,0.028595,0.0,0.020220,0.0,0.035021,0.028595,...,0.045212,0.053496,0.040439,0.0,0.000000,0.040439,0.000000,0.045212,0.000000,0.028595
3,"""Титан"", группа компаний (деревообработка)",140,0.0,0.0,0.000000,0.0,0.000000,0.0,0.020024,0.028318,...,0.040048,0.044775,0.044775,0.0,0.000000,0.034683,0.000000,0.044775,0.028318,0.000000
4,En+ Group,26,0.0,0.0,0.000000,0.0,0.018544,0.0,0.032119,0.026225,...,0.045423,0.026225,0.026225,0.0,0.018544,0.018544,0.000000,0.041466,0.000000,0.037088


In [ ]:
raex_list = (
    pd.read_excel("RAEX2024.xlsx")
    .drop(columns=['№', 'Код MOEX', 'Подотрасль', 'Год последней оцененной отчетности', 'url', 'url_sustainability'])
    .rename(columns={
        'Название': 'company',
        'ESG-рейтинг': 'ESG_Rating_band',
        'E Rank': 'E_Rank',
        'E-рейтинг': 'E_Rating_band',
        'S Rank': 'S_Rank',
        'S-рейтинг': 'S_Rating_band',
        'G Rank': 'G_Rank',
        'G-рейтинг': 'G_Rating_band',
    })
)

raex_list['E_Rank'] = raex_list['E_Rank'].astype('str')
raex_list['S_Rank'] = raex_list['S_Rank'].astype('str')
raex_list['G_Rank'] = raex_list['G_Rank'].astype('str')

raex_list.head()

,company,ESG_Rating_band,E_Rank,E_Rating_band,S_Rank,S_Rating_band,G_Rank,G_Rating_band
0,Сбербанк,AA,2,AA,1,AAA,4,AAA
1,«ФосАгро»,AA,5,AA,3,AA,2,AAA
2,«Полюс»,AA,1,AA,2,AA,14,AA
3,"«Татнефть», группа",AA,13,A,5,AA,6,AA
4,"«Роснефть», нефтяная компания",AA,4,AA,22,A,5,AAA


In [ ]:
features_data = features_data.merge(raex_list, on='company', how='left')
features_data

,company,ESG_Rank,topic_num_0,topic_num_1,topic_num_2,topic_num_3,topic_num_4,topic_num_5,topic_num_6,topic_num_7,...,topic_num_57,topic_num_58,topic_num_59,ESG_Rating_band,E_Rank,E_Rating_band,S_Rank,S_Rating_band,G_Rank,G_Rating_band
0,"""Каустик"" (г. Волгоград)",60,0.000000,0.0,0.034638,0.000000,0.000000,0.0,0.018972,0.015490,...,0.026830,0.010953,0.028980,B,72,CC,52,BB,55,BBB
1,"""Нефтиса"", нефтяная компания",144-145,0.000000,0.0,0.030994,0.000000,0.000000,0.0,0.030994,0.030994,...,0.030994,0.000000,0.030994,C,119,C,137,C,139,CC
2,"""Соликамскбумпром""",86,0.000000,0.0,0.028595,0.000000,0.020220,0.0,0.035021,0.028595,...,0.045212,0.000000,0.028595,CCC,88,CC,98,CC,75,BB
3,"""Титан"", группа компаний (деревообработка)",140,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.020024,0.028318,...,0.044775,0.028318,0.000000,C,123,C,139,C,133,CC
4,En+ Group,26,0.000000,0.0,0.000000,0.000000,0.018544,0.0,0.032119,0.026225,...,0.041466,0.000000,0.037088,BBB,26,BB,35,BBB,20,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,ТГК-2,111,0.000000,0.0,0.000000,0.000000,0.019353,0.0,0.033520,0.027369,...,0.033520,0.000000,0.043274,CC,121,C,105,CC,104,CCC
116,ТМК (Трубная металлургическая компания),44,0.018541,0.0,0.018541,0.026221,0.026221,0.0,0.026221,0.026221,...,0.032114,0.000000,0.018541,BB,57,CCC,49,BB,13,AA
117,Транспортная группа FESCO,89,0.000000,0.0,0.018897,0.013362,0.013362,0.0,0.023143,0.018897,...,0.029878,0.035352,0.013362,CCC,96,C,77,CCC,101,B
118,Южуралзолото Группа Компаний,138,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.027896,0.027896,...,0.039451,0.000000,0.039451,C,130,C,141,C,128-129,CC


In [ ]:
features_data.to_csv('features_data_train.csv', encoding='utf-8')